In [32]:
import pandas as pd
from datetime import datetime,timedelta

In [ ]:
df = pd.read_csv('data 4.csv', on_bad_lines='skip')
df

In [29]:
df = pd.read_csv('data 4.csv', encoding='latin-1')
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [30]:
df['TotalSum'] = df['Quantity'] * df['UnitPrice']
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalSum
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France,16.60


In [35]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
current_date = df['InvoiceDate'].max()
data_process = df.groupby(['CustomerID']).agg({
                'InvoiceDate': lambda x: (current_date - x.max()).days,
                'InvoiceNo': 'count',
                'TotalSum': 'sum'})
data_process.columns = ['Recency', 'Frequency', 'Monetary']
data_process

,Recency,Frequency,Monetary
CustomerID,,,
12346.0,325,2,0.00
12347.0,1,182,4310.00
12348.0,74,31,1797.24
12349.0,18,73,1757.55
12350.0,309,17,334.40
...,...,...,...
18280.0,277,10,180.60
18281.0,180,7,80.82
18282.0,7,13,176.60


In [36]:
data_process['rfm_score'] = data_process['Recency'] * 0.3 + data_process['Frequency'] * 0.4 + data_process['Monetary'] * 0.3
data_process

,Recency,Frequency,Monetary,rfm_score
CustomerID,,,,
12346.0,325,2,0.00,98.300
12347.0,1,182,4310.00,1366.100
12348.0,74,31,1797.24,573.772
12349.0,18,73,1757.55,561.865
12350.0,309,17,334.40,199.820
...,...,...,...,...
18280.0,277,10,180.60,141.280
18281.0,180,7,80.82,81.046
18282.0,7,13,176.60,60.280


In [40]:
import numpy as np

data_process['percent_off'] = 5  # Default value
data_process.loc[data_process['rfm_score'] > 100, 'percent_off'] = 10
data_process


,Recency,Frequency,Monetary,rfm_score,percent_off
CustomerID,,,,,
12346.0,325,2,0.00,98.300,5
12347.0,1,182,4310.00,1366.100,10
12348.0,74,31,1797.24,573.772,10
12349.0,18,73,1757.55,561.865,10
12350.0,309,17,334.40,199.820,10
...,...,...,...,...,...
18280.0,277,10,180.60,141.280,10
18281.0,180,7,80.82,81.046,5
18282.0,7,13,176.60,60.280,5


In [41]:
data_process.loc[data_process['rfm_score'] > 500, 'percent_off'] = 15
data_process.loc[data_process['rfm_score'] > 1000, 'percent_off'] = 20


In [42]:
data_process

,Recency,Frequency,Monetary,rfm_score,percent_off
CustomerID,,,,,
12346.0,325,2,0.00,98.300,5
12347.0,1,182,4310.00,1366.100,20
12348.0,74,31,1797.24,573.772,15
12349.0,18,73,1757.55,561.865,15
12350.0,309,17,334.40,199.820,10
...,...,...,...,...,...
18280.0,277,10,180.60,141.280,10
18281.0,180,7,80.82,81.046,5
18282.0,7,13,176.60,60.280,5


In [43]:
last_rows = df.groupby('InvoiceNo').tail(1)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalSum
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom,25.50
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom,11.10
20,536367,48187,DOORMAT NEW ENGLAND,4,2010-12-01 08:34:00,7.95,13047.0,United Kingdom,31.80
24,536368,22914,BLUE COAT RACK PARIS FASHION,3,2010-12-01 08:34:00,4.95,13047.0,United Kingdom,14.85
25,536369,21756,BATH BUILDING BLOCK WORD,3,2010-12-01 08:35:00,5.95,13047.0,United Kingdom,17.85
...,...,...,...,...,...,...,...,...,...
541866,581583,85038,6 CHOCOLATE LOVE HEART T-LIGHTS,36,2011-12-09 12:23:00,1.85,13777.0,United Kingdom,66.60
541868,581584,85038,6 CHOCOLATE LOVE HEART T-LIGHTS,48,2011-12-09 12:25:00,1.85,13777.0,United Kingdom,88.80
541889,581585,22466,FAIRY TALE COTTAGE NIGHT LIGHT,12,2011-12-09 12:31:00,1.95,15804.0,United Kingdom,23.40
541893,581586,20685,DOORMAT RED RETROSPOT,10,2011-12-09 12:49:00,7.08,13113.0,United Kingdom,70.80


In [44]:
def send_notfication(row):
  return f"Please complete transacton for {row['Description']}"

last_rows['notification'] = last_rows.apply(send_notfication, axis=1)
last_rows


<ipython-input-44-e3584e11338f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_rows['notification'] = last_rows.apply(send_notfication, axis=1)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalSum,notification
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom,25.50,Please complete transacton for GLASS STAR FROS...
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom,11.10,Please complete transacton for HAND WARMER RED...
20,536367,48187,DOORMAT NEW ENGLAND,4,2010-12-01 08:34:00,7.95,13047.0,United Kingdom,31.80,Please complete transacton for DOORMAT NEW ENG...
24,536368,22914,BLUE COAT RACK PARIS FASHION,3,2010-12-01 08:34:00,4.95,13047.0,United Kingdom,14.85,Please complete transacton for BLUE COAT RACK ...
25,536369,21756,BATH BUILDING BLOCK WORD,3,2010-12-01 08:35:00,5.95,13047.0,United Kingdom,17.85,Please complete transacton for BATH BUILDING B...
...,...,...,...,...,...,...,...,...,...,...
541866,581583,85038,6 CHOCOLATE LOVE HEART T-LIGHTS,36,2011-12-09 12:23:00,1.85,13777.0,United Kingdom,66.60,Please complete transacton for 6 CHOCOLATE LOV...
541868,581584,85038,6 CHOCOLATE LOVE HEART T-LIGHTS,48,2011-12-09 12:25:00,1.85,13777.0,United Kingdom,88.80,Please complete transacton for 6 CHOCOLATE LOV...
541889,581585,22466,FAIRY TALE COTTAGE NIGHT LIGHT,12,2011-12-09 12:31:00,1.95,15804.0,United Kingdom,23.40,Please complete transacton for FAIRY TALE COTT...
541893,581586,20685,DOORMAT RED RETROSPOT,10,2011-12-09 12:49:00,7.08,13113.0,United Kingdom,70.80,Please complete transacton for DOORMAT RED RET...
